# lending関係サービスのweb APIの試用

以下のサービスを試す。

| サービス名 | 状況 | 
| --- | --- |
| BASE | 審査待ち |
| マネーフォワード | 法人向けはまだAPIを公開してないため、後回し |
| SBIネット銀行 | 口座開設待ち |
| freee | OAuth2.0のやり取りにあたって、手元にwebサーバーが必要？らしく実験中 |
| zaim | 未 |


## [BASEのWeb API](https://github.com/baseinc/api-docs)の試用

アクセストークン取得のため、[こちらから](https://developers.thebase.in/)あらかじめ申請をする必要がある。
[利用規約](https://thebase.in/pages/api_term)に同意する必要がある。注意を引いた項目は以下の２つくらい

1. 自社のサービスでBASEへのリンクを貼って「使ってますよ」という事を宣伝する必要がある。
2. このAPIをBASEと競合するサービスのために使ってはならない。

ここで審査待ちが発生する。

pythonの[requests-oauthlibライブラリのOAuth2用公式ドキュメント](http://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html)を参考にする。


In [6]:
import requests
# from urllib.parse import
import webbrowser
from requests_oauthlib import OAuth1Session
from IPython.core.display import display, HTML

In [ ]:
baseurl = "https://api.thebase.in/"
client_id = ""
secret = ""

requests.get("https://api.github.com/user")

## マネーフォワード

SBI、MIZUHO、三井住友銀行といったサービスと連携しており、国内では最多らしい。
これら既存金融機関との連携にあたっては参照系API(GET)だけではなく更新系(POST)も最近始めたらしく、強い。

公開しているのはごく一部らしく、現時点では提携しないと使えない物が多い。
また、法人側(MFクラウド会計)はまだAPIを公開していない。
個人側(MFクラウド確定申告)は使えるようだが、とりあえず後回し

In [ ]:
url = "https://api.moneyforward.com/oauth/authorize"
res = requests.get(url)
print(res)

## 住信SBIネット銀行

APIの公開にあたって、IBMのサービスを利用している。Bluemixを使用しているのかと思ったが、オンプレミスでも使えるサービスらしい

口座開設待ち中

![](http://special.nikkeibp.co.jp/atcl/NBO/16/ibm0615/img1.png)

## [freee](https://secure.freee.co.jp/developers/api/doc?q=), [Zaim](https://dev.zaim.net/)

freeeのAPIはすでに[SHARESというサードパーティ企業によって利用されている](https://iotnews.jp/archives/11488)

OAuth2.0を使用

> (コールバックURIは)ローカル環境でのテストには `urn:ietf:wg:oauth:2.0:oob` をご利用ください。

とのこと


In [18]:
appid = "2e3a35b132e32628f617a197255e523f9cc55c18032f216c32c7470e001e2653"
secret = "d1de57fabbfb27bfa998dcb0f7bab0431de79d44c57a3823091f218bd2213b12"


def call_freee_api(uri, params):
    # url = "https://api.freee.co.jp"
    # auth_url = "https://secure.freee.co.jp/oauth/authorize"
    # user_url = url + relative_uri

    res = requests.get(url, params = params)
    res.raise_for_status()
    return res


authorization_code_flow_params = {
        "client_id" : appid,
        "client_secret": secret,
        "redirect_uri": "urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob",
        "response_type": "code"
}   
res = call_freee_api("/api/1/users/me.json", authorization_code_flow_params)
# print(res)
# display(HTML(res.text)) # 認証用widgetが表示されるが、手元にwebサーバーがないと動かない。
# with open("./freee_return.html", "w") as outfh:
    # outfh.write(res.text)

In [20]:
url = "https://secure.freee.co.jp/oauth/authorize"
implicit_flow_params = {
    "client_id": appid,
    "redirect_uri": "urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob",
    "response_type": "token",
    "client_secret": secret
}

res = call_freee_api(url, implicit_flow_params)
res.text

'<!DOCTYPE html>\n<html>\n<head>\n  <meta charset="UTF-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"7f45471ba9","applicationID":"2211425","transactionName":"JgwIREcJDVsGEEpBARYQD19bFU5ZBhU=","queueTime":0,"applicationTime":92,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VwIBWFBbGwECVVBWBQQ="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(15),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typ